<a href="https://colab.research.google.com/github/obinnachike/custom-bot-with-llama2/blob/main/Custom_bot_with_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers

In [ ]:
!pip -q install unstructured

In [ ]:
!pip install tokenizers

In [ ]:
!pip install xformers

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [ ]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [ ]:
data

In [ ]:
len(data)

In [ ]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [ ]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

In [ ]:
text_chunks

In [ ]:
text_chunks[0]

In [ ]:
text_chunks[1]

#**5: Download the Hugging Face Embeddings**

In [ ]:
#embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings=HuggingFaceEmbeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("How are you")
len(query_result)

In [ ]:
query_result

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [ ]:
!pip install faiss-cpu

In [ ]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [ ]:
index_name='llama'

In [ ]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [ ]:
# Save FAISS index to disk
vectorstore.save_local(index_name)


#**07: Create a Large Language Model (LLM) Wrapper**

In [ ]:
notebook_login()

In [ ]:
#model="meta-llama/Llama-2-7b-chat-hf"
model="daryl149/llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

#**08: Initialize the Retrieval QA with Source Chain**

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
query = "How good is Vicuna?"

In [ ]:
docs = vectorstore.similarity_search(query, k=3)

In [ ]:
docs

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [ ]:
query = "How good is Vicuna?"
qa.run(query)

In [ ]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")